In [1]:
import os
print(os.getcwd())
%load_ext autoreload
%autoreload 2

/home/udit/programs/Synaptic-Flow/Notebooks


In [2]:
import sys
sys.path.append("/home/udit/programs/Synaptic-Flow/")

In [6]:
args = {
"dataset" : "mnist", # ['mnist','cifar10','cifar100','tiny-imagenet','imagenet']
"model" : "fc", # ['fc','fc-orth','conv','conv-orth','strconv', ... <take rest from main.py>]
"model_class" : "default", # ['default','lottery','tinyimagenet','imagenet']
"dense_classifier" : False,
"pretrained" : False,
"optimizer" : "adam", # ['sgd','momentum','adam','rms']
"train_batch_size" : 128,
"test_batch_size" : 128,
"pre_epochs" : 0, # number of epochs to train before pruning
"post_epochs" : 20, # number of epochs to train after pruning
"lr" : 0.001,
"lr_drops" : [30, 60, 80],
"lr_drop_rate" : 0.1,
"weight_decay" : 1e-4 ,
"save" : False,
"scale" : 1,

# pruning args
"pruner" : "synflow", # ['rand','mag','snip','grasp','synflow']
"compression" : 1.0, # quotient of prunable non-zero prunable parameters before and after pruning (defaul"t: 1.0)
"prune_epochs" : 10, # number of iterations for scoring (defaul"t: 1)
"compression_schedule" : "exponential", # ['linear','exponential']
"mask_scope" : "global", # ['global','local']
"prune_dataset_ratio" : 10, # ratio of prune dataset size and number of classes (defaul"t: 10)'
"prune_batch_size" : 256, # input batch size for pruning (defaul"t: 256)
"prune_bias" : False,
"prune_batchnorm" : False,
"prune_residual" : False,
"reinitialize" : False, # whether to reinitialize weight parameters after pruning (defaul"t: False)
"pruner_list" : [], # list of pruning strategies for singleshot (defaul"t: [])
"prune_epoch_list" : [], # list of prune epochs for singleshot (defaul"t: [])
"compression_list" : [], # list of compression ratio exponents for singleshot/multishot (defaul"t: [])
"level_list" : [],
"experiment" : "prune-only", # ['example',"prune-only",'singleshot','multishot',
    # 'unit-conservation', 'layer-conservation','imp-conservation','schedule-conservation']
"expid" : "",
"result_dir" : "Results/data",
"gpu" : "0",
"workers" : 4,
"no_cuda" : False,
"seed" : 1,
"verbose" : False,
"save_pruned" : False,
"save_pruned_path" : "Results/pruned"
}

In [98]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from Utils import load
from Utils import generator
from Utils import metrics
from prune import *
from train import *
import matplotlib.pyplot as plt

import os

def prune_only(args):
    print("Pruning only")
    
    ## Random Seed and Device ##
    torch.manual_seed(args["seed"])
    device = load.device(args["gpu"])
    shuffle = False
    ## Data ##
    print('Loading {} dataset.'.format(args["dataset"]))
    input_shape, num_classes = load.dimension(args["dataset"]) 
    prune_loader = load.dataloader(args["dataset"], args["prune_batch_size"], True, args["workers"]) #, 
                                   # args["prune_dataset_ratio"] * num_classes)
    
    train_loader = load.dataloader(args["dataset"], args["train_batch_size"], False, args["workers"])
    test_loader = load.dataloader(args["dataset"], args["test_batch_size"], False, args["workers"])
    
    
    ## Model, Loss, Optimizer ##
    print('Creating {}-{} model.'.format(args["model_class"], args["model"]))        

    model = load.model(args["model"], args["model_class"])(input_shape, 
                                                     num_classes, 
                                                     args["dense_classifier"], 
                                                     args["pretrained"]).to(device)
    loss = nn.CrossEntropyLoss()
    
        
    opt_class, opt_kwargs = load.optimizer(args["optimizer"])
    optimizer = opt_class(generator.parameters(model), lr=args["lr"], 
                          weight_decay=args["weight_decay"], **opt_kwargs)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=args["lr_drops"], 
                                                     gamma=args["lr_drop_rate"])

    
    #print(f"Kernel: {kernel}")
    
    ## Pre-Train ##
    #print('Pre-Train for {} epochs.'.format(args.pre_epochs))
    pre_result = train_eval_loop(model, loss, optimizer, scheduler, train_loader, 
                                 test_loader, device, 0, args["verbose"])
    
    
    ## Prune ##
    print('Pruning with {} for {} epochs.'.format(args["pruner"], args["prune_epochs"]))
    pruner = load.pruner(args["pruner"])(generator.masked_parameters(model, args["prune_bias"], 
                       args["prune_batchnorm"], args["prune_residual"]))
    sparsity = 10**(-float(args["compression"]))
    print("Sparsity: {}".format(sparsity))
    save_pruned_path = args["save_pruned_path"] + "/%s/%s/%s" % (args["model_class"], 
                                                                 args["model"], args["pruner"],)
    if (args["save_pruned"]):
        print("Saving pruned models to: %s" % (save_pruned_path, ))
        if not os.path.exists(save_pruned_path):
            os.makedirs(save_pruned_path)
    prune_loop(model, loss, pruner, prune_loader, device, sparsity, 
               args["compression_schedule"], args["mask_scope"], args["prune_epochs"], 
               args["reinitialize"], args["save_pruned"], save_pruned_path)
    
    prune_result = metrics.summary(model, pruner.scores,
                                   metrics.flop(model, input_shape, device),
                                   lambda p: generator.prunable(p, args["prune_batchnorm"], 
                                    args["prune_residual"]))
    total_params = int((prune_result['sparsity'] * prune_result['size']).sum())
    possible_params = prune_result['size'].sum()
    total_flops = int((prune_result['sparsity'] * prune_result['flops']).sum())
    possible_flops = prune_result['flops'].sum()
    print("Parameter Sparsity: {}/{} ({:.4f})".format(total_params, 
                                                      possible_params, total_params / possible_params))
    print("FLOP Sparsity: {}/{} ({:.4f})".format(total_flops, 
                                                 possible_flops, total_flops / possible_flops))
    
    ## Post-Train ##
    #print('Post-Training for {} epochs.'.format(args.post_epochs))
    post_result = train_eval_loop(model, loss, optimizer, scheduler, train_loader, 
                                  test_loader, device, args["post_epochs"], args["verbose"])
    
    print(save_pruned_path)
    ## Display Results ##
#     frames = [pre_result.head(1), post_result.head(1), post_result.tail(1)]
#     train_result = pd.concat(frames, keys=['Init.', 'Post-Prune', "Final"])

#     print("Train results:\n", train_result)
#     print("Prune results:\n", prune_result)
    if (args["save_result"]):
        save_result_path = args["save_pruned_path"] + "/%s/%s/%s" % (args["model_class"], 
                                                                 args["model"], args["pruner"],)
        if not os.path.exists(save_pruned_path):
            os.makedirs(save_result_path)
        post_result.to_csv(save_result_path + "/%s" % (args["dataset"] + "_" + str(args["seed"]) 
                                                                        + "_" + str(args["compression"]) + ".csv"))
    ## Save Results and Model ##
    if args["save"]:
        print('Saving results.')
        pre_result.to_pickle("{}/pre-train.pkl".format(args["result_dir"]))
        post_result.to_pickle("{}/post-train.pkl".format(args["result_dir"]))
        prune_result.to_pickle("{}/compression.pkl".format(args["result_dir"]))
        torch.save(model.state_dict(),"{}/model.pt".format(args["result_dir"]))
        torch.save(optimizer.state_dict(),"{}/optimizer.pt".format(args["result_dir"]))
        torch.save(pruner.state_dict(),"{}/pruner.pt".format(args["result_dir"]))
    
    # to change
    return post_result

In [51]:
args['post_epochs'] = 30
args['prune_epochs'] = 100
args['model'] = "vgg11-bn"
args['dataset'] =  'cifar10' # 'tiny-imagenet'
args['lr'] = 0.0001
args['optimizer'] = 'adam'
args['model_class'] = 'lottery'
args['pruner'] = "synflow"
args['compression'] = 1.0
args['verbosse'] = True
prune_only(args)

Pruning only
Loading cifar10 dataset.
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Creating lottery-vgg11-bn model.


0it [00:00, ?it/s]
  0%|          | 0/100 [00:00<?, ?it/s]

Pruning with synflow for 100 epochs.
Sparsity: 0.1


100%|██████████| 100/100 [00:23<00:00,  4.28it/s]

Parameter Sparsity: 930550/9231114 (0.1008)
FLOP Sparsity: 41381662/153224202 (0.2701)



100%|██████████| 30/30 [02:02<00:00,  4.08s/it]


,train_loss,test_loss,top1_accuracy,top5_accuracy
0,NaN,2.311064,10.15,48.38
1,1.570313,1.213881,57.71,96.03
2,1.064090,0.869591,69.33,98.54
3,0.680498,0.632139,78.12,99.10
4,0.345361,0.593127,78.91,99.28
5,0.193451,1.273772,61.58,96.73
6,0.167934,0.436271,84.38,99.94
7,0.129766,0.109078,97.07,100.00
8,0.057542,0.081676,97.90,100.00
9,0.026889,0.044886,98.91,100.00


In [55]:
args['post_epochs'] = 30
args['prune_epochs'] = 100
args['model'] = "vgg11-bn"
args['dataset'] =  'cifar100' # 'tiny-imagenet'
args['lr'] = 0.001
args['optimizer'] = 'adam'
args['model_class'] = 'lottery'
args['pruner'] = "synflow-dist"
args['compression'] = 2.0
prune_only(args)

Pruning only
Loading cifar100 dataset.
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Creating lottery-vgg11-bn model.


0it [00:00, ?it/s]
  0%|          | 0/100 [00:00<?, ?it/s]

Pruning with synflow-dist for 100 epochs.
Sparsity: 0.01
Feeding image_mean
Calculating mean


100%|██████████| 100/100 [00:07<00:00, 12.66it/s]

Parameter Sparsity: 101045/9277284 (0.0109)
FLOP Sparsity: 7096682/153270372 (0.0463)



100%|██████████| 30/30 [02:02<00:00,  4.07s/it]


,train_loss,test_loss,top1_accuracy,top5_accuracy
0,NaN,4.605134,1.02,5.48
1,4.122572,4.133383,7.90,28.18
2,3.627807,3.659594,12.89,38.51
3,3.325873,3.256981,20.35,48.02
4,3.073351,3.174447,21.60,50.37
5,2.851671,3.011569,24.31,53.32
6,2.667793,2.935781,26.18,56.45
7,2.457366,3.007600,25.97,56.49
8,2.280894,2.480884,35.26,67.60
9,2.116923,2.480918,34.45,67.71


In [112]:
args['post_epochs'] = 20
args['prune_epochs'] = 100
args['model'] = "vgg11"
args['dataset'] =  'cifar10'
args['lr'] = 0.001
args['optimizer'] = 'adam'
args['model_class'] = 'lottery'
args['pruner'] = "synflow-dist"
args['compression'] = 1.5
args['seed'] = 42
args['save_result'] = True
prune_only(args)

Pruning only
Loading cifar10 dataset.
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Creating lottery-vgg11 model.


0it [00:00, ?it/s]
  0%|          | 0/100 [00:00<?, ?it/s]

Pruning with synflow-dist for 100 epochs.
Sparsity: 0.03162277660168379
Feeding image_mean
Calculating mean


100%|██████████| 100/100 [00:08<00:00, 12.47it/s]


Parameter Sparsity: 294415/9225610 (0.0319)
FLOP Sparsity: 17962473/152921098 (0.1175)


100%|██████████| 20/20 [01:15<00:00,  3.76s/it]

Results/pruned/lottery/vgg11/synflow-dist


,train_loss,test_loss,top1_accuracy,top5_accuracy
0,NaN,2.302206,9.93,50.79
1,2.164463,2.060572,20.56,78.50
2,1.948346,2.043545,25.89,83.74
3,1.762894,2.232344,27.01,83.61
4,1.644475,1.839692,30.62,89.02
5,1.549937,1.845559,33.40,84.54
6,1.468996,1.629057,37.55,90.73
7,1.378861,1.402449,48.23,93.41
8,1.309684,1.389588,49.60,94.30
9,1.211154,1.194031,54.51,95.76


In [109]:
args['post_epochs'] = 20
args['prune_epochs'] = 100
args['model'] = "vgg11"
args['dataset'] =  'cifar10'
args['lr'] = 0.001
args['optimizer'] = 'adam'
args['model_class'] = 'lottery'
args['pruner'] = "synflow"
args['compression'] = 1.0
args['save_result'] = True
prune_only(args)

Pruning only
Loading cifar10 dataset.
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Creating lottery-vgg11 model.


0it [00:00, ?it/s]
  0%|          | 0/100 [00:00<?, ?it/s]

Pruning with synflow for 100 epochs.
Sparsity: 0.1


100%|██████████| 100/100 [00:24<00:00,  4.02it/s]

Parameter Sparsity: 925046/9225610 (0.1003)
FLOP Sparsity: 41091425/152921098 (0.2687)



100%|██████████| 20/20 [01:15<00:00,  3.76s/it]

Results/pruned/lottery/vgg11/synflow


,train_loss,test_loss,top1_accuracy,top5_accuracy
0,NaN,2.310711,10.17,49.45
1,2.313412,2.141395,16.64,74.35
2,1.958337,1.972080,23.93,83.64
3,1.741563,1.682321,35.64,88.46
4,1.618244,1.623462,37.63,90.08
5,1.470046,1.419434,46.47,93.13
6,1.390034,1.349639,51.23,93.69
7,1.296421,1.661358,45.19,87.75
8,1.230554,1.307816,52.56,93.84
9,1.131306,1.113393,59.70,96.34


VGG11: 
- CIFAR 10 - adam, 0.001, 60/120, 0.1, 1e-4 both compression: 1 and 2, 100/160
- CIFAR 100 - adam/0.0005, mom/0.001, 60/120, 0.1, 1e-4 both compression: 1 and 2, 100/160
- TINYIMAGENET - 

VGG11-BN:
- CIFAR 10 - adam, 0.0001 (comp: 1), 0.0001 (comp: 2), 60/120, 0.1, 1e-4 100/160
- CIFAR 100 - adam, 0.0001 (comp: 1), 0.0005 (comp: 2), 60/120, 0.1, 1e-4 100/160

VGG16:
- CIFAR 10 - adam, 0.001, 60/120, 0.1, 1e-4, compression 1
- CIFAR 100 - adam, 0.0001, 60/120, 0.1, 1e-4, compression 1

VGG16-BN:
- CIFAR 10 - adam, 0.001 1/2 60/120, 0.1, 1e-4
- CIFAR 100 - adam, 0.001 (comp: 1)/ 0.0001 (comp: 2), 60/120, 0.1, 1e-4

Resnet20:
- CIFAR10 - adam 0.0005 (comp: 1), 0.0005 (comp: 2) 
- CIFAR100 - adam 0.01

Resnet18:
- Tiny Imagenet:

WideResnet20:
- CIFAR10 - adam 0.001 (comp: 1/2),
- CIFAR100 - adam 0.001 (comp: 1/2), 

WideResnet18:
- Tiny Imagenet: 